In [ ]:

import datetime
import tensorflow as tf
import matplotlib.pyplot as plt
import keras_ocr
import cv2 as cv
import math
import numpy as np
import skimage
from skimage.transform import  rescale,resize
from skimage.util import img_as_uint,img_as_ubyte
from skimage.color import rgb2gray
from skimage import exposure
import glob
from skimage.filters import threshold_yen

In [ ]:

def midpoint(x1, y1, x2, y2):
    x_mid = int((x1 + x2)/2)
    y_mid = int((y1 + y2)/2)
    return (x_mid, y_mid)


def remove_text(img_path):
    pipeline = keras_ocr.pipeline.Pipeline()

    # read image
    img = keras_ocr.tools.read(img_path)

    # generate (word, box) tuples 
    prediction_groups = pipeline.recognize([img])
    mask = np.zeros(img.shape[:2], dtype="uint8")

    for box in prediction_groups[0]:
        x0, y0 = box[1][0]
        x1, y1 = box[1][1] 
        x2, y2 = box[1][2]
        x3, y3 = box[1][3] 
        
        x_mid0, y_mid0 = midpoint(x1, y1, x2, y2)
        x_mid1, y_mi1 = midpoint(x0, y0, x3, y3)
        
        thickness = int(math.sqrt( (x2 - x1)**2 + (y2 - y1)**2 ))
        
        cv.line(mask, (x_mid0, y_mid0), (x_mid1, y_mi1), 255, thickness)
        removed_text_img = cv.inpaint(img, mask, 100, cv.INPAINT_NS)
    
    #cv.imshow("removed text", img)    

    return removed_text_img

In [ ]:
def change_contrast(img, cliplimit: float):
    bwimage = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    clahe = cv.createCLAHE(clipLimit=cliplimit, tileGridSize=(8, 8))
    contrast_image = clahe.apply(bwimage)
    return contrast_image



In [ ]:
def gammaCorrection(img, gamma):
    invGamma = 1 / gamma

    table = [((i / 255) ** invGamma) * 255 for i in range(256)]
    table = np.array(table, np.uint8)

    return cv.LUT(img, table)

In [ ]:
def sharpen(image):
  kernel = np.array([[0, -1, 0],
                [-1, 5,-1],
                [0, -1, 0]])
  image_sharp = cv.filter2D(src=image, ddepth=-1, kernel=kernel)


In [ ]:
def yen(img):
  # read image
  img = cv.imread(img)
  thresh = threshold_yen(img)
  # binary = img >= thresh
  return thresh

In [ ]:
def center_crop(img):
    dim = (250, 250)
    width, height = img.shape[1], img.shape[0]
    # process crop width and height for max available dimension
    crop_width = dim[0] if dim[0]<img.shape[1] else img.shape[1]
    crop_height = dim[1] if dim[1]<img.shape[0] else img.shape[0] 
    mid_x, mid_y = int(width/2), int(height/2)
    cw2, ch2 = int(crop_width/2), int(crop_height/2) 
    crop_img = img[mid_y-ch2:mid_y+ch2, mid_x-cw2:mid_x+cw2]
    return crop_img

In [ ]:
def scale_image(img, factor=1):
  #Returns resize image by scale factor
  return cv.resize(img,(int(img.shape[1]*factor), int(img.shape[0]*factor)))